## STEP 2: Data Pre-Processing

### Import necessary libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

### 2.1 Load datasets from file paths

In [ ]:
data_path = '..\Dataset\data.csv'
genre_data_path = '..\Dataset\data_by_genres.csv'

# Check if files exist and load them
if os.path.exists(data_path) and os.path.exists(genre_data_path):
    data = pd.read_csv(data_path)
    genre_data = pd.read_csv(genre_data_path)
    print("Info: Data and genre data successfully loaded.")
else:
    print("Attention: One or both files are not found in the specified directory.")

### 2.2 Descriptive statistics of features
This displays descriptive statistics for each dataset in a structured format.
Descriptive statistics (like mean, std, min, max, and quartiles) provide insights into data distributions.It helps in identifying central tendency, spread, and outliers across multiple datasets in one go.\
Explanation of descriptive statistics:
1. Count : The total number of non-missing entries for each feature.
2. Mean : The average value for each feature.
3. Std (Standard Deviation) : The spread or variability of the data around the mean.
4. Min : The minimum value in the dataset for each feature.
5. 25% (First Quartile) : The value below which 25% of the data falls.
6. 50% (Median) : The middle value, or the value below which 50% of the data falls.
7. 75% (Third Quartile) : The value below which 75% of the data falls.
8. Max : The maximum value in the dataset for each feature.

In [ ]:
# Summary Statistics for numerical columns
num_data = data.select_dtypes(include=['float64', 'int64'])
data_desc = num_data.describe().transpose()

num_genredata = genre_data.select_dtypes(include=['float64', 'int64'])
genredata_desc = num_genredata.describe().transpose()

print("Summary Statistics for Data:")
display(data_desc)
print("\nSummary Statistics for Genre Data:")
display(genredata_desc)

# Unique Values for each column
data_unique_val = {col: data[col].nunique() for col in data.columns}
print("\nUnique Values in Each Column of Data:")
display(data_unique_val)

genredata_unique_val = {col: genre_data[col].nunique() for col in genre_data.columns}
print("\nUnique Values in Each Column of Genre data:")
display(genredata_unique_val)

#### Conclusion for Descriptive statistics
1. **Valence, Acousticness, Danceability, Energy, and Key:** These features exhibit a balanced distribution between 0 and 1, with means around 0.5. This indicates a diverse representation of musical characteristics, suggesting a wide variety of tracks in the dataset.

2. **Duration (ms):** Converting the duration from milliseconds to minutes will enhance the understanding of trends related to track length and its potential influence on other features.

3. **Instrumentalness:** The distribution of instrumentalness is left-skewed, with very few tracks exceeding a value of 0.5. This suggests that the dataset primarily consists of tracks with significant vocal presence, which is typical in most music collections.

4. **Speechiness:** The low mean value of approximately 0.10 indicates that most tracks are primarily musical rather than speech-heavy, supporting the expectation for a music recommendation system that focuses on songs rather than podcasts.

5. **Popularity:** With 75% of the tracks having a popularity score below 48, there is a notable scarcity of highly popular tracks. This highlights an opportunity for the recommendation system to explore and promote lesser-known but potentially valuable tracks.

6. **Year :** We have data samples from 1921 to 2020 in this dataset.

7. **loudness :**  The loudness feature ranges from -60.0 to 3.855, encompassing both positive and negative values. To facilitate better analysis and ensure that all features are on a similar scale, it would be beneficial to normalize this feature.

### 2.3 Visualizing Outliers in Numerical Variables with Boxplots

Outliers are data samples that falls on the far left or right side of the ordered data. Generally, the outliers fall more than the specified distance from the first and third quartile (IQR: Interquartile Range),(i.e.) outliers are greater than Q3 + (1.5 * IQR) or less than Q1 - (1.5 * IQR).

In [ ]:
def plot_boxplots_with_outliers_summary(df, dataset_name):
    
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    num_cols = len(numerical_columns)
    rows = (num_cols // 3) + (num_cols % 3 > 0)  # Calculate number of rows needed for subplots
    outlier_counts = {}
    
    plt.figure(figsize=(15, rows * 5)) 
    
    for i, column in enumerate(numerical_columns):
        plt.subplot(rows, 3, i + 1)  
        sns.boxplot(y=df[column])
        plt.title(f'Box Plot for {column}')  
        plt.ylabel('Values')
        
        # Calculating IQR and identifying outliers
        Q1 = df[column].quantile(0.25)   # 25th precentile
        Q3 = df[column].quantile(0.75)   # 75th percentile
        IQR = Q3 - Q1      # length of the box (see box plot) IQR: Interquartile Range
        lower_bound = Q1 - 1.5 * IQR  # After this outliers are there (if any)
        upper_bound = Q3 + 1.5 * IQR   # After this outliers are there (if any)
        
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)] # To Find the outliers
        
        # Count outliers and add to the dictionary
        num_outliers = outliers.shape[0]
        outlier_counts[column] = num_outliers
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  
    plt.suptitle(f'Box Plots for {dataset_name}', fontsize=16, y=0.98)  
    plt.show()
    
    # SUMMARY
    print(f"Outlier Summary for {dataset_name}:")
    for column, count in outlier_counts.items():
        print(f"{column}: {count} outliers")

plot_boxplots_with_outliers_summary(data, 'Data')

#### Conclusion from Boxplot Analysis of Outliers

The columns duration_ms , instrumentalness, liveness, loudness, speechiness, tempo show values outside the whiskers of the boxplots, indicating potential outliers.

- **duration_ms and tempo**: Songs with very high durations (like live performances) or unique tempos (extremely slow or fast) are often valid data points and may be retained depending on analysis needs.
- **instrumentalness, liveness and speechiness** : Given that this is a music recommendation system prioritizing vocal songs over live recordings or podcasts, it is expected that the dataset will show a greater concentration of values near 0 for instrumentalness, liveness, and speechiness. This aligns with the intention to recommend tracks that are primarily music-focused and feature significant vocal elements.
- **loudness**: Extremely low loudness values may represent specific song genres or recording qualities and may need further investigation to decide if they should be capped or retained.
- **explicit** : This is a categorical feature and it has only few data samples with explicit feature value as 1.However, it might help in personalised song recommendations.Hence, its better to retain this feature.
- **danceability** : There are 143 outliers with low danceability which can be capped for better model performance.

### 2.4 Outlier Handling

For outlier handling , Outlier Capping is chosen for its simplicity, effectiveness, and ability to retain data integrity, providing a balanced approach to outlier handling. This method minimizes the impact of extreme values while preserving the dataset for accurate, interpretable analysis.

In [ ]:
def cap_outliers(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.25)         # 25th percentile (first quartile)
        Q3 = df[column].quantile(0.75)         # 75th percentile (third quartile)
        IQR = Q3 - Q1                          # Interquartile Range (IQR)
        lower_bound = Q1 - 1.5 * IQR           # Lower limit for outliers
        upper_bound = Q3 + 1.5 * IQR           # Upper limit for outliers
        
        # Apply capping: replace values outside bounds with nearest boundary values
        df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df

# Specify columns for outlier handling based on analysis
columns_to_cap_data = ['danceability']

data_cleaned = cap_outliers(data.copy(), columns_to_cap_data)

# Display summary statistics of cleaned data
display(data_cleaned[columns_to_cap_data].describe())

In [ ]:
#Plot the boxplots of all features after outlier capping

plot_boxplots_with_outliers_summary(data_cleaned, 'Data after Outlier capping')

#### Conclusion from Outliers handling

- The column danceability had 143 outliers , and after outlier capping , the column has no outlier as plotted in above boxplot.
- All other features are retained with outlier , as they have too many outliers and they can provide important information for music recommendation system.


### 2.5 Feature Engineering

We have added new columns Description,duration_min,loudness_scaled, release_decade, tempo_category to provide better interpretability, facilitate trend analysis, and allow easier integration with models that benefit from normalized or categorical data.


In [ ]:
data_feature_stats = data.describe().loc[['min', '25%', '50%', '75%', 'max']]

print('Important Data statistics')
display(data_feature_stats)

In [ ]:
expanded_interpretation_matrix = {
    "Feature": [],
    "Range": [],
    "Description": [],
    "Data Stats": [],
}

common_columns_stats = set(data.columns).intersection(set(genre_data.columns))

for column in common_columns_stats:
    for range_stat in ['min', '25%', '50%', '75%', 'max']:
        expanded_interpretation_matrix["Feature"].append(column)
        expanded_interpretation_matrix["Range"].append(range_stat)
        
        # Set interpretation based on each feature's meaning in the music industry
        if column == "valence":
            description = {
                'min': "Very Low - emotionally negative or neutral",
                '25%': "Low - somewhat negative or subdued emotion",
                '50%': "Moderate - balanced or neutral emotion",
                '75%': "High - positive and bright emotion",
                'max': "Very High - highly positive and happy"
            }
        elif column == "acousticness":
            description = {
                'min': "No acoustic elements, entirely synthetic or electronic",
                '25%': "Low acoustic presence, minor acoustic components",
                '50%': "Moderate acoustic presence, some mix of electronic and acoustic sounds",
                '75%': "High acoustic presence, mainly acoustic",
                'max': "Extremely acoustic, almost entirely devoid of electronic elements"
            }
        elif column == "danceability":
            description = {
                'min': "Very low danceability, not suited for dancing",
                '25%': "Low danceability, limited rhythmic appeal",
                '50%': "Moderate danceability, suitable for light dancing",
                '75%': "High danceability, good rhythmic appeal",
                'max': "Very high danceability, very rhythmic and suited for dancing"
            }
        elif column == "energy":
            description = {
                'min': "Very low energy, calm or subdued",
                '25%': "Low energy, relaxed or mellow",
                '50%': "Moderate energy, balanced intensity",
                '75%': "High energy, lively or intense",
                'max': "Very high energy, intense and energetic"
            }
        elif column == "liveness":
            description = {
                'min': "Very low, studio recording with no live elements",
                '25%': "Low live presence, possibly slight live recording",
                '50%': "Moderate live presence, hints of live performance",
                '75%': "High live presence, strong live recording elements",
                'max': "Very high, complete live recording or concert setting"
            }
        elif column == "instrumentalness":
            description = {
                'min': "No instrumental elements, entirely vocal",
                '25%': "Low instrumental presence, primarily vocal with some instrumental",
                '50%': "Moderate instrumental presence, significant instrumental parts",
                '75%': "High instrumental presence, mostly instrumental",
                'max': "Very high instrumental presence, entirely instrumental"
            }
        elif column == "speechiness":
            description = {
                'min': "Very low, mainly musical sounds",
                '25%': "Low presence of spoken words",
                '50%': "Moderate speech, balanced with music",
                '75%': "High speech, primarily vocal-driven",
                'max': "Very high, mainly spoken words or narration"
            }
        elif column == "tempo":
            description = {
                'min': "Very slow, potentially ambient or background music",
                '25%': "Slow tempo, relaxing and unhurried",
                '50%': "Moderate tempo, typical for pop and rock",
                '75%': "Fast tempo, upbeat and energetic",
                'max': "Very fast, high-energy music such as EDM"
            }
        elif column == "loudness":
            description = {
                'min': "Very quiet, often ambient or background sound",
                '25%': "Low loudness, soft or subdued sound",
                '50%': "Moderate loudness, typical for most music",
                '75%': "High loudness, strong sound presence",
                'max': "Very loud, intense and full-bodied sound"
            }
        elif column == "popularity":
            description = {
                'min': "Very low popularity, niche or little-known",
                '25%': "Low popularity, limited audience",
                '50%': "Moderately popular, some audience reach",
                '75%': "High popularity, well-liked and familiar",
                'max': "Very high popularity, widely known and favored"
            }
        elif column == "key":
            description = {
                'min': "Very low, typically quieter tonal center",
                '25%': "Low, less common tonal center",
                '50%': "Moderate, standard tonal center",
                '75%': "High, popular tonal centers like G or A",
                'max': "Very high, uncommon tonal centers"
            }
        elif column =="duration_ms":
            description = {
                'min': "Very short duration, often intros or interludes",
                '25%': "Short duration, typically for radio or pop songs",
                '50%': "Moderate duration, standard length for most songs",
                '75%': "Long duration, typical for extended versions or certain genres",
                'max': "Very long duration, usually instrumental or progressive pieces"
            }

        else:
            description = {stat: f"Interpretation for {column} at {stat}" for stat in ['min', '25%', '50%', '75%', 'max']}

        expanded_interpretation_matrix["Description"].append(description[range_stat])
        expanded_interpretation_matrix["Data Stats"].append(data_feature_stats.loc[range_stat, column])

expanded_interpretation_matrix_df = pd.DataFrame(expanded_interpretation_matrix)

display(expanded_interpretation_matrix_df)

In [ ]:
# DICTIONARY
category_thresholds = {
    "valence": [data_feature_stats["valence"]["min"], data_feature_stats["valence"]["25%"],
                data_feature_stats["valence"]["50%"], data_feature_stats["valence"]["75%"], data_feature_stats["valence"]["max"]],
    "acousticness": [data_feature_stats["acousticness"]["min"], data_feature_stats["acousticness"]["25%"],
                     data_feature_stats["acousticness"]["50%"], data_feature_stats["acousticness"]["75%"], data_feature_stats["acousticness"]["max"]],
    "danceability": [data_feature_stats["danceability"]["min"], data_feature_stats["danceability"]["25%"],
                     data_feature_stats["danceability"]["50%"], data_feature_stats["danceability"]["75%"], data_feature_stats["danceability"]["max"]],
    "energy": [data_feature_stats["energy"]["min"], data_feature_stats["energy"]["25%"],
               data_feature_stats["energy"]["50%"], data_feature_stats["energy"]["75%"], data_feature_stats["energy"]["max"]],
    "liveness": [data_feature_stats["liveness"]["min"], data_feature_stats["liveness"]["25%"],
                 data_feature_stats["liveness"]["50%"], data_feature_stats["liveness"]["75%"], data_feature_stats["liveness"]["max"]],
    "instrumentalness": [data_feature_stats["instrumentalness"]["min"], data_feature_stats["instrumentalness"]["25%"],
                         data_feature_stats["instrumentalness"]["50%"], data_feature_stats["instrumentalness"]["75%"], data_feature_stats["instrumentalness"]["max"]],
    "speechiness": [data_feature_stats["speechiness"]["min"], data_feature_stats["speechiness"]["25%"],
                    data_feature_stats["speechiness"]["50%"], data_feature_stats["speechiness"]["75%"], data_feature_stats["speechiness"]["max"]],
    "tempo": [data_feature_stats["tempo"]["min"], data_feature_stats["tempo"]["25%"],
              data_feature_stats["tempo"]["50%"], data_feature_stats["tempo"]["75%"], data_feature_stats["tempo"]["max"]],
    "loudness": [data_feature_stats["loudness"]["min"], data_feature_stats["loudness"]["25%"],
                 data_feature_stats["loudness"]["50%"], data_feature_stats["loudness"]["75%"], data_feature_stats["loudness"]["max"]],
    "popularity": [data_feature_stats["popularity"]["min"], data_feature_stats["popularity"]["25%"],
                   data_feature_stats["popularity"]["50%"], data_feature_stats["popularity"]["75%"], data_feature_stats["popularity"]["max"]],
    "key": [data_feature_stats["key"]["min"], data_feature_stats["key"]["25%"],
            data_feature_stats["key"]["50%"], data_feature_stats["key"]["75%"], data_feature_stats["key"]["max"]],
    "duration_ms": [data_feature_stats["duration_ms"]["min"], data_feature_stats["duration_ms"]["25%"],
                    data_feature_stats["duration_ms"]["50%"], data_feature_stats["duration_ms"]["75%"], data_feature_stats["duration_ms"]["max"]]
}

def categorize_feature(value, thresholds, descriptions):
    if value <= thresholds[1]:
        return descriptions[0]             # Min description
    elif value <= thresholds[2]:
        return descriptions[1]             # 25% description
    elif value <= thresholds[3]:
        return descriptions[2]             # 50% description
    elif value <= thresholds[4]:
        return descriptions[3]             # 75% description
    else:
        return descriptions[4]             # Max description

for feature, thresholds in category_thresholds.items():
    
    # Retrieve descriptions for this feature from the interpretation matrix
    descriptions = expanded_interpretation_matrix_df[expanded_interpretation_matrix_df["Feature"] == feature]["Description"].values
    
    # Apply categorization based on thresholds
    data[f"{feature}_category"] = data[feature].apply(lambda x: categorize_feature(x, thresholds, descriptions))

# Convert duration from milliseconds to minutes and discarding duration_ms column
data['duration_min'] = round(data['duration_ms'] / 60000)
data = data.drop(columns=['duration_ms'])

# Add column loudness_Scaled with normalized loudness
data['loudness_scaled'] = (data['loudness'] - data['loudness'].min()) / (data['loudness'].max() - data['loudness'].min())

# Extract the release decade from the year
data['release_decade'] = (data['year'] // 10) * 10

# Add column 'mode_category' using mode 
data['mode_category'] = data['mode'].map({0: 'Low, often minor scale (sadder tone)', 1: 'High, often major scale (happier tone)'})

# Add column 'explicit_category' using explicit 
data['explicit_category'] = np.where(data['explicit'] == 1, 'Yes', 'No')

print("Data after adding new columns")
display(data.head())

In [ ]:
#Remove  release_date column as its incomplete and does not provide any information
data = data.drop(columns=['release_date'])
display(data.columns)

#### Conclusion

- To address the negative values in the 'loudness' column, we created a new column, 'loudness_scaled', which has been normalized to a range of 0 to 1.
- The column 'duration_min' is added after converting 'duration_ms' to minutes and duration_ms is discared.
- The column 'release_decade' is added after converting 'year' to corresponding discarded.
- Category Columns 'mode_category', 'explicit_category' are added which helps in mapping numerical values i.e. 0 and 1 to the corresponding meaning.This will help in further data analysis.
- Columns'valence_category', 'acousticness_category', 'danceability_category','energy_category', 'liveness_category', 'instrumentalness_category',
  'speechiness_category', 'tempo_category', 'loudness_category','popularity_category', 'key_category', 'duration_ms_category' have been added to provide a clearer understanding of the numerical values, based on their relative positions within the overall range (e.g., less than 25%, 50%, 75%, or 100%).

### 2.6 Store the pre-processed dataset

In [ ]:
# Save the raw datasets as raw.csv files
data.to_csv('..\Dataset\data_cleaned.csv', index=False)  